In [17]:
import os
import numpy as np
import tensorflow as tf
from tqdm import tqdm

# Importa as funções e classes do seu código local
from src.datasets import MSMarcoDataset
from src.retrivers.bm25 import BM25
from src.metrics import mmr_score, ndcg_score

# Carrega o dataset (ajuste os caminhos conforme necessário)
dataset = MSMarcoDataset('data/subset_msmarco_train_0')
dataset.get_data('subset_msmarco_train_0.01_99.pkl')

# Inicializa o BM25 com os documentos
bm25 = BM25(dataset.documents)

print("Dataset carregado e BM25 inicializado!")


Loading data from subset_msmarco_train_0.01_99.pkl...


Loading qrels: 100%|██████████| 2845/2845 [00:00<00:00, 6432.71it/s]
[nltk_data] Downloading package punkt_tab to /Users/masfz/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to /Users/masfz/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Tokenizing data...


Tokenizing documents: 100%|██████████| 277168/277168 [00:40<00:00, 6911.38it/s]


Split data into train and test sets...
Data loading and processing complete.
Dataset carregado e BM25 inicializado!


In [18]:
from transformers import BertTokenizer, TFBertForSequenceClassification

# Usando o modelo pré-treinado "bert-base-uncased" como exemplo.
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
monobert_model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')

print("Modelo monoBERT (bert-base-uncased) e tokenizer carregados com sucesso!")

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Modelo monoBERT (bert-base-uncased) e tokenizer carregados com sucesso!


In [19]:
def score_with_monobert(query, doc_texts, max_length=512):
    """
    Para cada par (query, documento), gera uma pontuação de relevância usando o monoBERT.
    
    Args:
        query (str): Texto da consulta.
        doc_texts (list of str): Lista contendo os textos dos documentos.
        max_length (int): Comprimento máximo dos tokens (default=512).
    
    Returns:
        scores (list of float): Lista de pontuações para cada documento.
    """
    scores = []
    for doc in doc_texts:
        # Prepara a entrada concatenando a query e o documento.
        inputs = tokenizer.encode_plus(
            query,
            doc,
            add_special_tokens=True,
            max_length=max_length,
            truncation=True,
            padding='max_length',
            return_tensors="tf"
        )
        # Executa o modelo em modo inference
        outputs = monobert_model(inputs)
        logits = outputs.logits  # formato: [batch_size, num_labels]
        # Supondo que a classe 1 representa relevância
        relevance_score = logits[0][1].numpy()  
        scores.append(relevance_score)
    return scores


In [21]:
# Defina um query_id para exemplificar (ajuste conforme seu dataset)
query_id = '135841'

# Recupera os candidatos usando BM25 (por exemplo, os top 100)
# bm25_results agora é uma lista de tuplas: (doc_id, BM25_score)
bm25_results = bm25.get_top_n(dataset.queries[query_id].tokenized_text, 100)

# Obtém os textos dos documentos correspondentes extraindo o doc_id de cada tupla
bm25_doc_texts = [dataset.documents[doc_id].text for doc_id, score in bm25_results]

# Recupera o texto completo da query
query_text = dataset.queries[query_id].text

# Calcula as pontuações de relevância usando monoBERT
monobert_scores = score_with_monobert(query_text, bm25_doc_texts)

# Cria uma lista apenas com os doc_ids (primeiro elemento de cada tupla)
doc_ids = [doc_id for doc_id, score in bm25_results]

# Combina os IDs dos documentos com as pontuações do monoBERT
doc_score_pairs = list(zip(doc_ids, monobert_scores))

# Reordena os documentos de acordo com a pontuação (maior indica maior relevância)
doc_score_pairs_sorted = sorted(doc_score_pairs, key=lambda x: x[1], reverse=True)

# Seleciona os top 10 documentos após reordenação
top10_monobert = doc_score_pairs_sorted[:10]

print("Top 10 documentos reordenados (BM25 + monoBERT):")
for rank, (doc_id, score) in enumerate(top10_monobert, start=1):
    print(f"Rank {rank}: Documento ID {doc_id} | Pontuação: {score:.4f}")


Top 10 documentos reordenados (BM25 + monoBERT):
Rank 1: Documento ID msmarco_passage_04_195360786 | Pontuação: 0.4171
Rank 2: Documento ID msmarco_passage_04_172245248 | Pontuação: 0.3788
Rank 3: Documento ID msmarco_passage_04_80690346 | Pontuação: 0.3760
Rank 4: Documento ID msmarco_passage_04_262374007 | Pontuação: 0.3679
Rank 5: Documento ID msmarco_passage_00_145991273 | Pontuação: 0.3637
Rank 6: Documento ID msmarco_passage_01_501767873 | Pontuação: 0.3546
Rank 7: Documento ID msmarco_passage_01_139428489 | Pontuação: 0.3519
Rank 8: Documento ID msmarco_passage_00_218383589 | Pontuação: 0.3457
Rank 9: Documento ID msmarco_passage_03_751267657 | Pontuação: 0.3448
Rank 10: Documento ID msmarco_passage_00_624439849 | Pontuação: 0.3399


In [25]:
# Célula de Visualização: Exibe a query e os top 10 documentos com seus textos e pontuações

# Usando o mesmo query_id de exemplo
query_id = '135841'
query_text = dataset.queries[query_id].text

# Recupera os candidatos via BM25 (top 100) e seus textos, calculando as pontuações do monoBERT
bm25_results = bm25.get_top_n(dataset.queries[query_id].tokenized_text, 100)
bm25_doc_texts = [dataset.documents[doc_id].text for doc_id, _ in bm25_results]
monobert_scores = score_with_monobert(query_text, bm25_doc_texts)
doc_ids = [doc_id for doc_id, _ in bm25_results]
doc_score_pairs = list(zip(doc_ids, monobert_scores))
doc_score_pairs_sorted = sorted(doc_score_pairs, key=lambda x: x[1], reverse=True)
top10_monobert = doc_score_pairs_sorted[:10]

# Exibe a query e os top 10 documentos com um trecho do texto
print("Query:", query_text)
print("\nTop 10 documentos reordenados (BM25 + monoBERT):")
for rank, (doc_id, score) in enumerate(top10_monobert, start=1):
    # Obtem o texto completo do documento e limita a um snippet (ex: 300 caracteres)
    doc_text = dataset.documents[doc_id].text
    snippet = doc_text[:300] + "..." if len(doc_text) > 300 else doc_text
    print(f"Rank {rank}: Documento ID {doc_id} | Pontuação: {score:.4f}")
    print("Trecho do documento:", snippet)
    print("-" * 80)


Query: definition of legal term injunctive relief

Top 10 documentos reordenados (BM25 + monoBERT):
Rank 1: Documento ID msmarco_passage_04_195360786 | Pontuação: 0.4171
Trecho do documento: What program did Roosevelt create to end the Great Depression. ? The New Deal was a set of economic programs, passed by Congress during President Roosevelt's first term, 1933-1937, in response to the Great Depression. The programs of The New Deal programs were to provide relief, recovery, and reform...
--------------------------------------------------------------------------------
Rank 2: Documento ID msmarco_passage_04_172245248 | Pontuação: 0.3788
Trecho do documento: What is the definition of melody melody? the definition of melody is a rhythm or something like a beat. :)
No, that's Rhythm, melody is the tune of a piece of music. Is there an easy definition of a dotted rhythm in music?
--------------------------------------------------------------------------------
Rank 3: Documento ID msmarco_

In [24]:
# Célula 7: Avaliação com Métricas (MRR e NDCG) usando a arquitetura Multirank
sorted_docs = {}

# Percorre cada consulta do conjunto de teste (isso pode demorar, pois invoca o modelo BERT para cada consulta)
for q_id in tqdm(dataset.test_query_ids, desc="Reordenando queries com monoBERT"):
    # Recupera os candidatos via BM25 (por exemplo, os top 100)
    bm25_results = bm25.get_top_n(dataset.queries[q_id].tokenized_text, 100)
    # bm25_results é uma lista de tuplas (doc_id, BM25_score)
    # Extraímos os textos dos documentos
    bm25_doc_texts = [dataset.documents[doc_id].text for doc_id, score in bm25_results]
    # Recupera o texto da consulta
    query_txt = dataset.queries[q_id].text
    # Calcula as pontuações de relevância com monoBERT para cada candidato
    monobert_scores = score_with_monobert(query_txt, bm25_doc_texts)
    # Extrai apenas os doc_ids
    doc_ids = [doc_id for doc_id, score in bm25_results]
    # Associa cada doc_id à pontuação do monoBERT
    doc_score_pairs = list(zip(doc_ids, monobert_scores))
    # Ordena os documentos em ordem decrescente de pontuação
    doc_score_pairs_sorted = sorted(doc_score_pairs, key=lambda x: x[1], reverse=True)
    # Seleciona os top 10 documentos para a consulta
    sorted_docs[q_id] = [doc_id for doc_id, score in doc_score_pairs_sorted[:10]]

# Calcula as métricas usando as funções importadas
mrr = mmr_score(sorted_docs, dataset.qrels)
ndcg = ndcg_score(sorted_docs, dataset.qrels)
print(f"MRR: {mrr:.4f}")
print(f"NDCG: {ndcg:.4f}")

Reordenando queries com monoBERT:   4%|▍         | 21/555 [16:35<7:01:48, 47.39s/it] 


KeyboardInterrupt: 